In [1]:
import numpy as np
import pandas as pd
import json
import datetime
import glob
import gc
import os

In [2]:
folder="/home/jian/Projects/Big_Lots/Weather/Json_data/daily/api_response_20180524/"
json_list_all=glob.glob(folder+"*.json")

In [3]:
inclusion_stores=pd.read_excel("/home/jian/Projects/Big_Lots/Q1_Post/BL_Sales YoY_JL_20180514.xlsx",sheetname="Inclusion Stores",skiprows=1,dtype=str)
store_zips=pd.read_excel("/home/jian/Projects/Big_Lots/Other_Input/all_store_DMA.xlsx",dtype=str)[['location_id','zip']]
store_zips['zip']=store_zips['zip'].apply(lambda x: x.zfill(5))
inclusion_stores=pd.merge(inclusion_stores,store_zips,on="location_id",how="left")
inclusion_stores=inclusion_stores[['location_id','zip']].rename(columns={"zip":"zip_cd"})
inclusion_stores=inclusion_stores[inclusion_stores['location_id']!='1769']

In [4]:
df_files=pd.DataFrame({"file":json_list_all},index=range(len(json_list_all)))
df_files['Date']=df_files['file'].apply(lambda x: datetime.datetime.strptime(x[len(x)-15:len(x)-5],"%Y-%m-%d"))
df_files=df_files[(df_files['Date']>=datetime.datetime(2018,2,4)) & (df_files['Date']<=datetime.datetime(2018,5,5))]

In [5]:
all_weather_groups=[]
all_weather_desc=[]
all_weather_id=[]
df_missing_weather=pd.DataFrame()
for file in df_files['file']:
    response=json.load(open(file,"r"))
    for zip_cd in inclusion_stores['zip_cd']:
        try:        
            data_zip_weather=response[zip_cd]['weather']
            for i in range(len(data_zip_weather)):            
                weather_group=data_zip_weather[i]['main']
                weather_desc=data_zip_weather[i]['description']
                
                all_weather_groups=list(set(all_weather_groups+[weather_group]))
                all_weather_desc=list(set(all_weather_desc+[weather_desc]))
                
        except:
            df_missing_weather=df_missing_weather.append(pd.DataFrame({"Date":file[len(file)-15:len(file)-5],"zips":zip_cd},index=[0]))
            # print("zip data not available: " + zip_cd+" | "+file[len(file)-15:len(file)-5])

In [6]:
df_missing_weather=df_missing_weather.sort_values(["Date","zips"])
writer_folder="/home/jian/Projects/Big_Lots/Weather/Q1_Weather_Counts/"+str(datetime.datetime.now().now().date())+"/"

try:
    os.stat(writer_folder)
except:
    os.makedirs(writer_folder)


writer=pd.ExcelWriter(writer_folder+"Q1_inclusion_store_all_weather_type_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

In [7]:
pd.DataFrame({"all_type_group":all_weather_groups}).to_excel(writer,"all_weather_group_list",index=False)
pd.DataFrame({"all_type_desc":all_weather_desc}).to_excel(writer,"all_weather_desc_list",index=False)
df_missing_weather.to_excel(writer,"missing_zips",index=False)

In [8]:
writer.save()

In [9]:
ranked_rain_snow_df=pd.read_excel("/home/jian/Projects/Big_Lots/Weather/Q1_Weather_Counts/Q1_inclusion_store_all_weather_type_ranked_Server.xlsx",sheetname="ranked rain snow")

ranked_rain=ranked_rain_snow_df.iloc[:,[4,5]]
ranked_rain.columns=[['rain_desc','rank']]
ranked_rain=ranked_rain.sort_values("rank")

ranked_snow=ranked_rain_snow_df.iloc[:,[0,1]]
ranked_snow.columns=[['snow_desc','rank']]
ranked_snow=ranked_snow[~pd.isnull(ranked_snow['snow_desc'])]
ranked_snow=ranked_snow.sort_values("rank")

In [10]:
ranked_rain.head(2)

,rain_desc,rank
0,light intensity drizzle rain,1
1,light intensity drizzle,1


# Temp, Rain Desc, Snow Desc, All Weather Group

In [11]:
df_output_rain=pd.DataFrame(columns=["zip_cd","Date","Collected","No_Rain"]+ranked_rain['rain_desc'].tolist())
df_output_snow=pd.DataFrame(columns=["zip_cd","Date","Collected","No_Snow"]+ranked_snow['snow_desc'].tolist())
df_output_temp=pd.DataFrame(columns=["zip_cd","Date","Collected","Temp","Temp_Max","Temp_Min"])
df_output_group=pd.DataFrame(columns=["zip_cd","Date","Collected"]+all_weather_groups)

In [12]:
k_count=0
for file in df_files['file']:
    date=datetime.datetime.strptime(file[len(file)-15:len(file)-5],"%Y-%m-%d").date()
    response=json.load(open(file,"r"))
        
    for zip_cd in inclusion_stores['zip_cd'].unique().tolist():
        df_rain=pd.DataFrame(columns=["zip_cd","Date","Collected","No_Rain"]+ranked_rain['rain_desc'].tolist(),index=[0])
        df_snow=pd.DataFrame(columns=["zip_cd","Date","Collected","No_Snow"]+ranked_snow['snow_desc'].tolist(),index=[0])
        df_temp=pd.DataFrame(columns=["zip_cd","Date","Collected","Temp","Temp_Max","Temp_Min"],index=[0])
        
        df_group=pd.DataFrame(columns=["zip_cd","Date","Collected"]+all_weather_groups,index=[0])
        
        
        df_rain['zip_cd']=zip_cd
        df_snow['zip_cd']=zip_cd
        df_temp['zip_cd']=zip_cd
        df_group['zip_cd']=zip_cd

        df_rain['Date']=date
        df_snow['Date']=date
        df_temp['Date']=date
        df_group['Date']=date
                                       
                                       
                                       
        
        if zip_cd in list(response.keys()):
            df_rain["Collected"]="Recorded"
            df_snow["Collected"]="Recorded"
            df_temp["Collected"]="Recorded"
            df_group["Collected"]="Recorded"
            
            temp=response[zip_cd]['main']['temp']*9/5 - 459.67
            temp_Max=response[zip_cd]['main']['temp_max']*9/5 - 459.67
            temp_Min=response[zip_cd]['main']['temp_min']*9/5 - 459.67

            df_temp['Temp']=temp
            df_temp['Temp_Max']=temp_Max
            df_temp['Temp_Min']=temp_Min

            for desc_rain in ranked_rain['rain_desc'].tolist():
                for j in range(len(response[zip_cd]['weather'])):
                    if response[zip_cd]['weather'][j]['description']==desc_rain:
                        df_rain[desc_rain]=1
            for desc_snow in ranked_snow['snow_desc'].tolist():
                for j in range(len(response[zip_cd]['weather'])):
                    if response[zip_cd]['weather'][j]['description']==desc_snow:
                        df_snow[desc_snow]=1
            for w_group in all_weather_groups:
                for j in range(len(response[zip_cd]['weather'])):
                    if response[zip_cd]['weather'][j]['main']==w_group:
                        df_group[w_group]=1
        else:
            df_rain["Collected"]="Not_recorded"
            df_snow["Collected"]="Not_recorded"
            df_temp["Collected"]="Not_recorded"
            df_group["Collected"]="Not_recorded"
            
        df_output_rain=df_output_rain.append(df_rain)
        df_output_snow=df_output_snow.append(df_snow)
        df_output_temp=df_output_temp.append(df_temp) 
        df_output_group=df_output_group.append(df_group)
    k_count=k_count+1
    print(k_count, datetime.datetime.now(),"finished of the date: ",date)


1 2018-06-15 12:54:53.562853 finished of the date:  2018-02-22
2 2018-06-15 12:55:10.471384 finished of the date:  2018-03-19
3 2018-06-15 12:55:28.353622 finished of the date:  2018-02-18
4 2018-06-15 12:55:47.503212 finished of the date:  2018-04-27
5 2018-06-15 12:56:08.811191 finished of the date:  2018-04-07
6 2018-06-15 12:56:31.170083 finished of the date:  2018-03-03
7 2018-06-15 12:56:54.415408 finished of the date:  2018-03-15
8 2018-06-15 12:57:18.629805 finished of the date:  2018-04-11
9 2018-06-15 12:57:45.192664 finished of the date:  2018-02-14
10 2018-06-15 12:58:13.032214 finished of the date:  2018-02-15
11 2018-06-15 12:58:41.920075 finished of the date:  2018-04-10
12 2018-06-15 12:59:12.659729 finished of the date:  2018-03-14
13 2018-06-15 12:59:45.546037 finished of the date:  2018-03-02
14 2018-06-15 13:00:19.001828 finished of the date:  2018-04-06
15 2018-06-15 13:00:53.825515 finished of the date:  2018-03-22
16 2018-06-15 13:01:28.999425 finished of the dat

In [13]:
df_output_rain['sum']=df_output_rain[ranked_rain['rain_desc'].tolist()].sum(axis=1)
df_output_snow['sum']=df_output_snow[ranked_snow['snow_desc'].tolist()].sum(axis=1)
df_output_rain['No_Rain']=np.where(pd.isnull(df_output_rain['sum']),1,np.nan)
df_output_snow['No_Snow']=np.where(pd.isnull(df_output_snow['sum']),1,np.nan)

df_output_rain['No_Rain']=np.where(df_output_rain['Collected']=="Not_recorded",np.nan,df_output_rain['No_Rain'])
df_output_snow['No_Snow']=np.where(df_output_snow['Collected']=="Not_recorded",np.nan,df_output_snow['No_Snow'])

del df_output_rain['sum']
del df_output_snow['sum']

In [14]:
df_output_temp_store=df_output_temp.copy()
df_output_temp_store=pd.merge(df_output_temp_store,inclusion_stores,on="zip_cd",how="left")

df_output_rain_store=df_output_rain.copy()
df_output_rain_store=pd.merge(df_output_rain_store,inclusion_stores,on="zip_cd",how="left")

df_output_snow_store=df_output_snow.copy()
df_output_snow_store=pd.merge(df_output_snow_store,inclusion_stores,on="zip_cd",how="left")

df_output_group_store=df_output_group.copy()
df_output_group_store=pd.merge(df_output_group_store,inclusion_stores,on="zip_cd",how="left")

In [15]:
writer=pd.ExcelWriter(writer_folder+"Q1_Snow_Rain_Desc_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
# df_output_temp_store.to_excel(writer,"Temp",index=False)
# df_output_rain_store.to_excel(writer,"Rain",index=False)
# df_output_snow_store.to_excel(writer,"Snow",index=False)
# df_output_group_store.to_excel(writer,"Group",index=False)
# writer.save()
# save later after get the avg difference

# Moving Average Tempreture

In [16]:
folder="/home/jian/Projects/Big_Lots/Weather/Json_data/daily/api_response_20180524/"
json_list_all=glob.glob(folder+"*.json")

df_files_avg_temp=pd.DataFrame({"file":json_list_all},index=range(len(json_list_all)))
df_files_avg_temp['Date']=df_files_avg_temp['file'].apply(lambda x: datetime.datetime.strptime(x[len(x)-15:len(x)-5],"%Y-%m-%d"))
df_files_avg_temp=df_files_avg_temp[(df_files_avg_temp['Date']>=datetime.datetime(2018,2,4)-datetime.timedelta(days=14)) & (df_files_avg_temp['Date']<=datetime.datetime(2018,5,5)+datetime.timedelta(days=14))]

In [17]:
df_output_temp_avg=pd.DataFrame(columns=["zip_cd","Date","Collected","Temp","Temp_Max","Temp_Min"])
k_count=0
for file in df_files_avg_temp['file']:
    date=datetime.datetime.strptime(file[len(file)-15:len(file)-5],"%Y-%m-%d").date()
    response=json.load(open(file,"r"))
        
    for zip_cd in inclusion_stores['zip_cd'].unique().tolist():
        df_temp_avg=pd.DataFrame(columns=["zip_cd","Date","Collected","Temp","Temp_Max","Temp_Min"],index=[0])
        df_temp_avg['zip_cd']=zip_cd
        df_temp_avg['Date']=date
        
        if zip_cd in list(response.keys()):
            df_temp_avg["Collected"]="Recorded"
            
            temp=response[zip_cd]['main']['temp']*9/5 - 459.67
            temp_Max=response[zip_cd]['main']['temp_max']*9/5 - 459.67
            temp_Min=response[zip_cd]['main']['temp_min']*9/5 - 459.67

            df_temp_avg['Temp']=temp
            df_temp_avg['Temp_Max']=temp_Max
            df_temp_avg['Temp_Min']=temp_Min

        else:
            df_temp_avg["Collected"]="Not_recorded"

        df_output_temp_avg=df_output_temp_avg.append(df_temp_avg)
    k_count=k_count+1
    print(k_count,datetime.datetime.now(),"finished of the date: ",date)
                

1 2018-06-15 15:20:10.428053 finished of the date:  2018-02-22
2 2018-06-15 15:20:14.626936 finished of the date:  2018-01-24
3 2018-06-15 15:20:18.868339 finished of the date:  2018-03-19
4 2018-06-15 15:20:23.196550 finished of the date:  2018-02-18
5 2018-06-15 15:20:27.637010 finished of the date:  2018-04-27
6 2018-06-15 15:20:32.168912 finished of the date:  2018-04-07
7 2018-06-15 15:20:36.748828 finished of the date:  2018-03-03
8 2018-06-15 15:20:41.290279 finished of the date:  2018-03-15
9 2018-06-15 15:20:45.952536 finished of the date:  2018-01-28
10 2018-06-15 15:20:50.636525 finished of the date:  2018-04-11
11 2018-06-15 15:20:55.361571 finished of the date:  2018-02-14
12 2018-06-15 15:21:00.132767 finished of the date:  2018-05-06
13 2018-06-15 15:21:04.952514 finished of the date:  2018-02-02
14 2018-06-15 15:21:09.824042 finished of the date:  2018-02-03
15 2018-06-15 15:21:14.754695 finished of the date:  2018-05-07
16 2018-06-15 15:21:19.746058 finished of the dat

In [18]:
df_output_temp_avg=df_output_temp_avg.sort_values(["zip_cd","Date"])

df_output_temp_avg_agg=pd.DataFrame()

for zip_cd,group in df_output_temp_avg.groupby(['zip_cd']):
    df=group.copy()
    df.reset_index()
    i=0
    for date_Q1 in df_files['Date']:
        df_day=df[(df['Date']>=date_Q1.date()-datetime.timedelta(days=14)) & (df['Date']<=date_Q1.date()+datetime.timedelta(days=14))]
        df_day_avg_temp=df_day['Temp'].mean()
        df_day_avg_temp_max=df_day['Temp_Max'].mean()
        df_day_avg_temp_min=df_day['Temp_Min'].mean()
        df_app=pd.DataFrame({"zip_cd":zip_cd,"Date":date_Q1.date(),"Avg_Temp":df_day_avg_temp,
                            "Avg_Temp_Max":df_day_avg_temp_max,"Avg_Temp_Min":df_day_avg_temp_min},index=[i])
        i=i+1
        
        df_output_temp_avg_agg=df_output_temp_avg_agg.append(df_app)        

In [19]:
df_output_temp_store=pd.merge(df_output_temp_store,df_output_temp_avg_agg,on=["Date","zip_cd"],how="left")
df_output_temp_store['Diff_Temp']=df_output_temp_store['Temp']-df_output_temp_store['Avg_Temp']
df_output_temp_store['Diff_Temp_Max']=df_output_temp_store['Temp_Max']-df_output_temp_store['Avg_Temp_Max']
df_output_temp_store['Diff_Temp_Min']=df_output_temp_store['Temp_Min']-df_output_temp_store['Avg_Temp_Min']

In [20]:
df_output_temp_store.to_excel(writer,"Temp",index=False)
df_output_rain_store.to_excel(writer,"Rain",index=False)
df_output_snow_store.to_excel(writer,"Snow",index=False)
df_output_group_store.to_excel(writer,"Group",index=False)
writer.save()

# Apply the Severity Directly

In [21]:
rain_snow_weight=pd.read_excel("/home/jian/Projects/Big_Lots/Weather/Q1_Weather_Counts/Q1_inclusion_store_all_weather_type_ranked.xlsx",sheetname="ranked rain snow")

rain_weight=rain_snow_weight.iloc[:,[4,6]]
rain_weight.columns=[['rain_desc','weight']]
rain_weight['weight']=rain_weight['weight'].astype(int)
rain_weight=rain_weight.sort_values("weight")

snow_weight=rain_snow_weight.iloc[:,[0,2]]
snow_weight.columns=[['snow_desc','weight']]
snow_weight=snow_weight[~pd.isnull(snow_weight['snow_desc'])]
snow_weight['weight']=snow_weight['weight'].astype(int)
snow_weight=snow_weight.sort_values("weight")

group_weight=pd.read_excel("/home/jian/Projects/Big_Lots/Weather/Q1_Weather_Counts/Q1_inclusion_store_all_weather_type_ranked.xlsx",sheetname="all_weather_group_list")
group_weight['Severity']=group_weight['Severity'].astype(int)
group_weight=group_weight[['all_type_group','Severity']]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [22]:
group_weight_dict=group_weight[['all_type_group', 'Severity']].set_index('all_type_group').T.to_dict()
rain_weight_dict=rain_weight[['rain_desc', 'weight']].set_index('rain_desc').T.to_dict()
snow_weight_dict=snow_weight[['snow_desc', 'weight']].set_index('snow_desc').T.to_dict()
rain_weight_dict.update({"No_Rain":{"weight":0}})
snow_weight_dict.update({"No_Snow":{"weight":0}})

In [23]:
for col in list(group_weight_dict.keys()):
    df_output_group_store[col]=df_output_group_store[col]*group_weight_dict[col]['Severity']
for col in list(rain_weight_dict.keys()):
    df_output_rain_store[col]=df_output_rain_store[col]*rain_weight_dict[col]['weight']
for col in list(snow_weight_dict.keys()):
    df_output_snow_store[col]=df_output_snow_store[col]*snow_weight_dict[col]['weight']
    
df_output_group_store['Severity']=df_output_group_store[list(group_weight_dict.keys())].max(axis=1)
df_output_rain_store['Severity']=df_output_rain_store[list(rain_weight_dict.keys())].max(axis=1)
df_output_snow_store['Severity']=df_output_snow_store[list(snow_weight_dict.keys())].max(axis=1)

In [24]:
'''
df_output_rain_store['Severity'][df_output_rain_store['Collected']=="Not_recorded"]="Not_recorded" or np.nan
df_output_snow_store['Severity'][df_output_snow_store['Collected']=="Not_recorded"]="Not_recorded" or np.nan
df_output_group_store['Severity'][df_output_group_store['Collected']=="Not_recorded"]="Not_recorded" or np.nan
'''

'\ndf_output_rain_store[\'Severity\'][df_output_rain_store[\'Collected\']=="Not_recorded"]="Not_recorded" or np.nan\ndf_output_snow_store[\'Severity\'][df_output_snow_store[\'Collected\']=="Not_recorded"]="Not_recorded" or np.nan\ndf_output_group_store[\'Severity\'][df_output_group_store[\'Collected\']=="Not_recorded"]="Not_recorded" or np.nan\n'

In [25]:
writer=pd.ExcelWriter(writer_folder+"Q1_Snow_Rain_Severity_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

df_output_temp_store.to_excel(writer,"Temp",index=False)
df_output_rain_store.to_excel(writer,"Rain",index=False)
df_output_snow_store.to_excel(writer,"Snow",index=False)
df_output_group_store.to_excel(writer,"Group",index=False)
writer.save()

In [26]:
df_output_temp_store.head(2)

,zip_cd,Date,Collected,Temp,Temp_Max,Temp_Min,location_id,Avg_Temp,Avg_Temp_Max,Avg_Temp_Min,Diff_Temp,Diff_Temp_Max,Diff_Temp_Min
0,43232,2018-02-22,Recorded,46.004,48.2,44.6,1,44.580071,46.592857,42.414286,1.423929,1.607143,2.185714
1,30906,2018-02-22,Recorded,79.502,80.6,77.0,3,66.513071,69.221429,63.757143,12.988929,11.378571,13.242857


# Weighted by Daily Traffic

In [27]:
# Abandoned to use the other method: consistant weight by weekday over the Q1

In [28]:
'''
traffic_folder="/home/jian/Projects/Big_Lots/Weather/Q1_Weather_Counts/Daily Traffic/"
traffic_files=glob.glob(traffic_folder+"*.txt")

file=traffic_files[0]
traffic_1=pd.read_csv(file,sep="|")
traffic_Q1=pd.DataFrame()
for file in traffic_files:
    df=pd.read_csv(file,sep="|",dtype=str)
    df['week_end_dt']=df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
    df['weekly_traffic']=0
    col_date_list=[]
    for i in range(1,8):
        day_num=str(i)
        col=[x for x in df.columns.tolist() if day_num in x][0]
        df[col]=df[col].astype(float)
        df['weekly_traffic']=df['weekly_traffic']+df[col]
        df_app=df[["location_id","week_end_dt"]+[col]]
        df_app.columns=["location_id","week_end_dt","Traffic"]
        df_app['Date']=df_app['week_end_dt'].unique()[0]-datetime.timedelta(days=(7-i))
        traffic_Q1=traffic_Q1.append(df_app)     

traffic_Q1_0=traffic_Q1[traffic_Q1['Traffic']==0]
weight_0=pd.DataFrame()
for store,group in traffic_Q1_0.groupby('location_id'):
    exclude_week=group['week_end_dt'].unique().tolist()
    df=traffic_Q1[(traffic_Q1['location_id']==store) & (~traffic_Q1['week_end_dt'].isin(exclude_week))]
    df['weekday']=df['Date'].apply(lambda x: x.weekday())
    df=df.groupby(["location_id","weekday"])['Traffic'].mean().to_frame().reset_index()
    sum_traffic=df['Traffic'].sum()
    df['Weight_by_Traffic']=df['Traffic'].apply(lambda x: x/sum_traffic)
    weight_0=weight_0.append(df)
del weight_0['Traffic']


def weighted_severity(df):
    result=pd.merge(df,traffic_Q1,on=["location_id","Date"],how="left")
    result=pd.merge(result,recorded_traffic_weight,on=["location_id","week_end_dt"],how="left")
    result['week_end_date_str']=result['week_end_dt'].astype(str)
    result['key']=result['location_id']+"|"+result['week_end_date_str']

    result_0=result[result['Traffic']==0]

    
    
    result_1=result[~result['key'].isin(result_0['key'])]
    result_1['Weight_by_Traffic']=result_1['Traffic']/result_1['weekly_traffic']
    result_1_col=result_1.columns.tolist()
    
    df_0=result[result['key'].isin(result_0['key'])]
    df_0['weekday']=df_0['Date'].apply(lambda x:x.weekday())
    df_0=pd.merge(df_0,weight_0,on=["location_id","weekday"],how="left")
    df_0=df_0[result_1_col]
    result_1['Weight_by_Traffic']=result_1['Traffic']/result_1['weekly_traffic']
    result=result_1.append(df_0)
    
    result['Weighted_Severity']=result['Severity']*result['Weight_by_Traffic']
    
    del result['week_end_date_str']
    return result
df_output_rain_store_weight=weighted_severity(df_output_rain_store)
df_output_snow_store_weight=weighted_severity(df_output_snow_store)
df_output_group_store_weight=weighted_severity(df_output_group_store)


df_output_temp_store_weight=df_output_temp_store.copy()
df_output_temp_store_weight['Date_str']=df_output_temp_store_weight['Date'].astype(str)
df_output_temp_store_weight['key']=df_output_temp_store_weight['location_id']+"|"+df_output_temp_store_weight['Date_str']
key_weight=df_output_snow_store_weight[['Date','location_id',"Traffic",'weekly_traffic','Weight_by_Traffic']]
key_weight['Date_str']=key_weight['Date'].astype(str)
key_weight['key']=key_weight['location_id']+"|"+key_weight['Date_str']
del key_weight['Date_str']
del key_weight['Date']
del key_weight['location_id']
df_output_temp_store_weight=pd.merge(df_output_temp_store_weight,key_weight,on="key",how="left")
del df_output_temp_store_weight['key']
df_output_temp_store_weight['Weighted_Diff_Temp']=df_output_temp_store_weight['Diff_Temp']*df_output_temp_store_weight['Weight_by_Traffic']
df_output_temp_store_weight['Weighted_Diff_Temp_Max']=df_output_temp_store_weight['Diff_Temp_Max']*df_output_temp_store_weight['Weight_by_Traffic']
df_output_temp_store_weight['Weighted_Diff_Temp_Min']=df_output_temp_store_weight['Diff_Temp_Min']*df_output_temp_store_weight['Weight_by_Traffic']


weighted_writer=pd.ExcelWriter(writer_folder+"Q1_Weighted_Snow_Rain_Desc_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_temp_store_weight.to_excel(weighted_writer,"Weighted_Temp",index=False)
df_output_rain_store_weight.to_excel(weighted_writer,"Weighted_Rain",index=False)
df_output_snow_store_weight.to_excel(weighted_writer,"Weighted_Snow",index=False)
df_output_group_store_weight.to_excel(weighted_writer,"Weighted_Group",index=False)
weighted_writer.save()
'''

'\ntraffic_folder="/home/jian/Projects/Big_Lots/Weather/Q1_Weather_Counts/Daily Traffic/"\ntraffic_files=glob.glob(traffic_folder+"*.txt")\n\nfile=traffic_files[0]\ntraffic_1=pd.read_csv(file,sep="|")\ntraffic_Q1=pd.DataFrame()\nfor file in traffic_files:\n    df=pd.read_csv(file,sep="|",dtype=str)\n    df[\'week_end_dt\']=df[\'week_end_dt\'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())\n    df[\'weekly_traffic\']=0\n    col_date_list=[]\n    for i in range(1,8):\n        day_num=str(i)\n        col=[x for x in df.columns.tolist() if day_num in x][0]\n        df[col]=df[col].astype(float)\n        df[\'weekly_traffic\']=df[\'weekly_traffic\']+df[col]\n        df_app=df[["location_id","week_end_dt"]+[col]]\n        df_app.columns=["location_id","week_end_dt","Traffic"]\n        df_app[\'Date\']=df_app[\'week_end_dt\'].unique()[0]-datetime.timedelta(days=(7-i))\n        traffic_Q1=traffic_Q1.append(df_app)     \n\ntraffic_Q1_0=traffic_Q1[traffic_Q1[\'Traffic\']==0]\nw

# Q1 Weekday weight applied

In [29]:
inclusion_YoY=pd.read_excel("/home/jian/Projects/Big_Lots/Q1_Post/BL_Sales YoY_JL_20180514.xlsx",sheetname="Inclusion Stores",skiprows=1,dtype=str)
inclusion_YoY=inclusion_YoY.iloc[:,0:29]

for col in inclusion_YoY.columns.tolist()[3:]:
    inclusion_YoY[col]=inclusion_YoY[col].astype(float)
inclusion_YoY['Sales_2017']=inclusion_YoY[inclusion_YoY.columns.tolist()[3:16]].sum(axis=1)
inclusion_YoY['Sales_2018']=inclusion_YoY[inclusion_YoY.columns.tolist()[16:29]].sum(axis=1)
inclusion_YoY['YoY']=(inclusion_YoY['Sales_2018']-inclusion_YoY['Sales_2017'])/inclusion_YoY['Sales_2017']
inclusion_YoY=inclusion_YoY[['location_id','DMA','Q1 Campaign Type','Sales_2017','Sales_2018','YoY']]

In [30]:
traffic_folder="/home/jian/Projects/Big_Lots/Weather/Q1_Weather_Counts/Daily Traffic/"
traffic_files=glob.glob(traffic_folder+"*.txt")

traffic_Q1=pd.DataFrame()

for file in traffic_files:
    df=pd.read_csv(file,sep="|",dtype=str)
    df['week_end_dt']=df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
    df['weekly_traffic']=0
    col_date_list=[]
    for i in range(1,8):
        day_num=str(i)
        col=[x for x in df.columns.tolist() if day_num in x][0]
        df[col]=df[col].astype(float)
        df['weekly_traffic']=df['weekly_traffic']+df[col]
        df_app=df[["location_id","week_end_dt"]+[col]]
        df_app['weekday']=col
        df_app.columns=["location_id","week_end_dt","Traffic","weekday"]
        df_app['Date']=df_app['week_end_dt'].unique()[0]-datetime.timedelta(days=(7-i))
        traffic_Q1=traffic_Q1.append(df_app)
traffic_Q1=traffic_Q1[traffic_Q1['Traffic']>0]
df_date_weekday=traffic_Q1[['Date','weekday']].drop_duplicates()
traffic_Q1=traffic_Q1[traffic_Q1['location_id'].isin(inclusion_YoY['location_id'])]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [31]:
df_weight_weekday_store=traffic_Q1.groupby(['location_id','weekday'])['Traffic'].mean().to_frame().reset_index()
df_weight_store=df_weight_weekday_store.groupby(['location_id'])['Traffic'].sum().to_frame().reset_index()
df_weight_store=df_weight_store.rename(columns={"Traffic":"weekly_traffic"})
df_weight_weekday_store=pd.merge(df_weight_weekday_store,df_weight_store,on="location_id",how="left")
df_weight_weekday_store['weight']=df_weight_weekday_store['Traffic']/df_weight_weekday_store['weekly_traffic']
df_weight_weekday_store['weekday_py']=np.where(df_weight_weekday_store['weekday']=="traffic_day_1",6,
                                               df_weight_weekday_store['weekday'].apply(lambda x: int(x[len(x)-1:])-2)                                              
                                              )
overall_weekday=traffic_Q1.groupby(['weekday'])['Traffic'].mean().to_frame().reset_index()
overall_weekday['weight']=overall_weekday['Traffic'].apply(lambda x: x/overall_weekday['Traffic'].sum())


overall_weekday['weekday_py']=np.where(overall_weekday['weekday']=="traffic_day_1",6,
                                               overall_weekday['weekday'].apply(lambda x: int(x[len(x)-1:])-2)                                              
                                              )



In [32]:
def weather_desc_by_day_table(df_input,desc_list):
    df_snow_desc_long=pd.DataFrame()
    for col in desc_list:
        df=df_input[['location_id','zip_cd','Date','Collected','Severity']+[col]]
        df=df.rename(columns={col:"nan_index"})
        df['desc']=col
        df_snow_desc_long=df_snow_desc_long.append(df)
    df_snow_desc_long_Not_Record=df_snow_desc_long[df_snow_desc_long['Collected']=="Not_recorded"][['location_id','zip_cd','Date']].drop_duplicates()
    df_snow_desc_long_no_nan=df_snow_desc_long[~pd.isnull(df_snow_desc_long['nan_index'])]

    df_snow_desc_long_Not_Record['key']=df_snow_desc_long_Not_Record['location_id']+"|"+df_snow_desc_long_Not_Record['Date'].apply(lambda x: str(x-datetime.timedelta(days=1)))
    df_snow_desc_long_Not_Record_Key=df_snow_desc_long_no_nan.copy()
    df_snow_desc_long_Not_Record_Key['key']=df_snow_desc_long_Not_Record_Key['location_id']+"|"+df_snow_desc_long_Not_Record_Key['Date'].apply(lambda x: str(x))
    df_snow_desc_long_Not_Record_Fill=df_snow_desc_long_Not_Record_Key[df_snow_desc_long_Not_Record_Key['key'].isin(df_snow_desc_long_Not_Record['key'])]
    del df_snow_desc_long_Not_Record_Fill['key']
    df_snow_desc_long_Not_Record_Fill['Date']=df_snow_desc_long_Not_Record_Fill['Date'].apply(lambda x: x+datetime.timedelta(days=1))

    df_snow_desc_long=df_snow_desc_long_no_nan.append(df_snow_desc_long_Not_Record_Fill)
    df_snow_desc_long['weekday_py']=df_snow_desc_long['Date'].apply(lambda x: x.weekday())
    
    df_snow_desc_long_in=pd.merge(df_snow_desc_long,df_weight_weekday_store[['location_id','weekday_py','weight']],
                                  on=['location_id','weekday_py'],how="left")
    df_snow_desc_long_out=df_snow_desc_long_in[pd.isnull(df_snow_desc_long_in['weight'])]
    # 4012 only
    df_snow_desc_long_in=df_snow_desc_long_in[~pd.isnull(df_snow_desc_long_in['weight'])]
    del df_snow_desc_long_out['weight']
    df_snow_desc_long_out=pd.merge(df_snow_desc_long_out,overall_weekday[['weekday_py','weight']],
                                  on=['weekday_py'],how="left")
    
    result=df_snow_desc_long_in.append(df_snow_desc_long_out)
    result['weighted_severity']=result['Severity']*result['weight']
    del result['nan_index']
    del result['weekday_py']
    return result

In [33]:
desc_list_snow=['No_Snow']+ranked_snow['snow_desc'].tolist()
df_snow_desc_long=weather_desc_by_day_table(df_output_snow_store,desc_list_snow)

desc_list_rain=['No_Rain']+ranked_rain['rain_desc'].tolist()
df_rain_desc_long=weather_desc_by_day_table(df_output_rain_store,desc_list_rain)

desc_list_group=all_weather_groups
df_weather_group_long=weather_desc_by_day_table(df_output_group_store,desc_list_group)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [34]:
df_weather_group_long.head(2)

,location_id,zip_cd,Date,Collected,Severity,desc,weight,weighted_severity
0,391,14043,2018-02-22,Recorded,5.0,Snow,0.125000,0.625000
1,819,14424,2018-02-22,Recorded,5.0,Snow,0.124264,0.621318


In [36]:

weather_group_rank=pd.read_excel("/home/jian/Projects/Big_Lots/Weather/Q1_Weather_Counts/weather_desc_drop_dup_rank.xlsx",sheetname="weather_group_rank")
rain_desc_rank=pd.read_excel("/home/jian/Projects/Big_Lots/Weather/Q1_Weather_Counts/weather_desc_drop_dup_rank.xlsx",sheetname="rain_desc_rank")
snow_desc_rank=pd.read_excel("/home/jian/Projects/Big_Lots/Weather/Q1_Weather_Counts/weather_desc_drop_dup_rank.xlsx",sheetname="snow_desc_rank")

df_rain_desc_long_dedup=pd.merge(df_rain_desc_long,rain_desc_rank,on="desc",how='left')
df_snow_desc_long_dedup=pd.merge(df_snow_desc_long,snow_desc_rank,on="desc",how='left')
df_weather_group_long_dedup=pd.merge(df_weather_group_long,weather_group_rank,on="desc",how='left')

df_rain_desc_long_dedup=df_rain_desc_long_dedup.sort_values('rank',ascending=False).drop_duplicates(['location_id','Date'])
df_snow_desc_long_dedup=df_snow_desc_long_dedup.sort_values('rank',ascending=False).drop_duplicates(['location_id','Date'])
df_weather_group_long_dedup=df_weather_group_long_dedup.sort_values('rank',ascending=False).drop_duplicates(['location_id','Date'])

df_rain_desc_long_dedup.to_csv(writer_folder+"df_rain_desc_long_by_day.csv",index=False)
df_snow_desc_long_dedup.to_csv(writer_folder+"df_snow_desc_long_by_day.csv",index=False)
df_weather_group_long_dedup.to_csv(writer_folder+"df_weather_group_long_by_day.csv",index=False)

In [44]:
df_weather_group_long_dedup['zip_cd']=df_weather_group_long_dedup['zip_cd'].apply(lambda x: x.zfill(5))
df_rain_desc_long_dedup['zip_cd']=df_rain_desc_long_dedup['zip_cd'].apply(lambda x: x.zfill(5))
df_snow_desc_long_dedup['zip_cd']=df_snow_desc_long_dedup['zip_cd'].apply(lambda x: x.zfill(5))


df_weather_group_long_dedup['weight']=df_weather_group_long_dedup['weight'].astype(float)
df_rain_desc_long_dedup['weight']=df_rain_desc_long_dedup['weight'].astype(float)
df_snow_desc_long_dedup['weight']=df_snow_desc_long_dedup['weight'].astype(float)

df_weather_group_long_dedup['weighted_severity']=df_weather_group_long_dedup['weighted_severity'].astype(float)
df_rain_desc_long_dedup['weighted_severity']=df_rain_desc_long_dedup['weighted_severity'].astype(float)
df_snow_desc_long_dedup['weighted_severity']=df_snow_desc_long_dedup['weighted_severity'].astype(float)

df_date_week_end=traffic_Q1[['week_end_dt','Date']].drop_duplicates()



In [45]:
inclusion_YoY_long=pd.read_excel("/home/jian/Projects/Big_Lots/Q1_Post/BL_Sales YoY_JL_20180514.xlsx",sheetname="Inclusion Stores",skiprows=1,dtype=str)
inclusion_YoY_long=inclusion_YoY_long.iloc[:,0:29]
inclusion_long_sales=pd.DataFrame()
for col in inclusion_YoY_long.columns.tolist()[3:]:
    inclusion_YoY_long[col]=inclusion_YoY_long[col].astype(float)
    df_long=inclusion_YoY_long[['location_id','DMA','Q1 Campaign Type']+[col]]
    df_long=df_long.rename(columns={col:"sales"})
    df_long['week_end_dt']=datetime.datetime.strptime(col,"%Y-%m-%d").date()
    inclusion_long_sales=inclusion_long_sales.append(df_long)
inclusion_long_sales_2017=inclusion_long_sales[inclusion_long_sales['week_end_dt']<=datetime.date(2017,12,1)]
inclusion_long_sales_2017=inclusion_long_sales_2017.rename(columns={"sales":"sales_weekly_2017"})
inclusion_long_sales_2017['week_end_dt']=inclusion_long_sales_2017['week_end_dt'].apply(lambda x: x+datetime.timedelta(days=52*7))
inclusion_long_sales_2018=inclusion_long_sales[inclusion_long_sales['week_end_dt']>=datetime.date(2017,12,1)]
inclusion_long_sales_2018=inclusion_long_sales_2018.rename(columns={"sales":"sales_weekly_2018"})

inclusion_long_sales=pd.merge(inclusion_long_sales_2017,inclusion_long_sales_2018,on=["location_id",'DMA','Q1 Campaign Type','week_end_dt'],how="left")

inclusion_long_sales=inclusion_long_sales[['location_id','DMA','Q1 Campaign Type','week_end_dt','sales_weekly_2017','sales_weekly_2018']]
inclusion_long_sales=pd.merge(df_date_week_end,inclusion_long_sales,on="week_end_dt",how="left")


In [46]:
df_rain_desc_long_dedup=pd.merge(df_rain_desc_long_dedup,inclusion_long_sales,on=["Date","location_id"],how="left")
df_snow_desc_long_dedup=pd.merge(df_snow_desc_long_dedup,inclusion_long_sales,on=["Date","location_id"],how="left")
df_weather_group_long_dedup=pd.merge(df_weather_group_long_dedup,inclusion_long_sales,on=["Date","location_id"],how="left")

In [47]:
store_week_count=df_weather_group_long_dedup.groupby(['location_id','week_end_dt'])['Date'].count().to_frame().reset_index()
store_week_count=store_week_count.rename(columns={"Date":"Recorded_Days"})


In [48]:
weather_group_P_95_peak_days=[datetime.date(2018,2,10),datetime.date(2018,2,24),datetime.date(2018,3,24),
                          datetime.date(2018,4,7),datetime.date(2018,4,8)]
weather_group_P_5_peak_days=[datetime.date(2018,3,3),datetime.date(2018,3,24),
                          datetime.date(2018,4,7),datetime.date(2018,4,8),datetime.date(2018,4,21)]
rain_desc_P_95_peak_days=[datetime.date(2018,2,10),datetime.date(2018,2,24),datetime.date(2018,3,24),
                          datetime.date(2018,4,7),datetime.date(2018,4,21)]
rain_desc_P_5_peak_days=[datetime.date(2018,3,3),datetime.date(2018,3,31),datetime.date(2018,4,7),
                          datetime.date(2018,4,8),datetime.date(2018,4,21)]
snow_desc_P_95_peak_days=[datetime.date(2018,2,17),datetime.date(2018,2,24),datetime.date(2018,3,24),
                          datetime.date(2018,4,7),datetime.date(2018,4,8)]
snow_desc_P_5_peak_days=[datetime.date(2018,3,10),datetime.date(2018,4,7),datetime.date(2018,4,8),
                          datetime.date(2018,4,20),datetime.date(2018,4,21)]

In [49]:
df_rain_desc_long_dedup=pd.merge(df_rain_desc_long_dedup,store_week_count,on=["location_id","week_end_dt"],how="left")
df_snow_desc_long_dedup=pd.merge(df_snow_desc_long_dedup,store_week_count,on=["location_id","week_end_dt"],how="left")
df_weather_group_long_dedup=pd.merge(df_weather_group_long_dedup,store_week_count,on=["location_id","week_end_dt"],how="left")

df_rain_desc_long_dedup['weighted_2_severity']=df_rain_desc_long_dedup['weighted_severity']/df_rain_desc_long_dedup['Recorded_Days']*7
df_snow_desc_long_dedup['weighted_2_severity']=df_snow_desc_long_dedup['weighted_severity']/df_snow_desc_long_dedup['Recorded_Days']*7
df_weather_group_long_dedup['weighted_2_severity']=df_weather_group_long_dedup['weighted_severity']/df_weather_group_long_dedup['Recorded_Days']*7

df_rain_desc_long_dedup['weighted_2_rank']=df_rain_desc_long_dedup['rank']/df_rain_desc_long_dedup['Recorded_Days']*7*df_rain_desc_long_dedup['weight']
df_snow_desc_long_dedup['weighted_2_rank']=df_snow_desc_long_dedup['rank']/df_snow_desc_long_dedup['Recorded_Days']*7*df_snow_desc_long_dedup['weight']
df_weather_group_long_dedup['weighted_2_rank']=df_weather_group_long_dedup['rank']/df_weather_group_long_dedup['Recorded_Days']*7*df_weather_group_long_dedup['weight']


In [111]:
df_weather_group_long_dedup.head(2)

,location_id,zip_cd,Date,Collected,Severity,desc,weight,weighted_severity,rank,week_end_dt,DMA,Q1 Campaign Type,sales_weekly_2017,sales_weekly_2018,Recorded_Days,weighted_2_severity,weighted_2_rank
0,391,14043,2018-02-22,Recorded,5.0,Snow,0.125000,0.62500,12,2018-02-24,BUFFALO,Balance,64079.62,82102.04,7,0.62500,1.500000
1,4636,60107,2018-03-06,Recorded,5.0,Snow,0.110874,0.55437,12,2018-03-10,CHICAGO,Balance,43586.96,47875.77,7,0.55437,1.330489


In [51]:
def agg_to_weekly(df):
    df_weighted_rank=df.groupby(["location_id","week_end_dt"])['weighted_2_rank'].sum().to_frame().reset_index()
    df_weighted_severity=df.groupby(["location_id","week_end_dt"])['weighted_2_severity'].sum().to_frame().reset_index()
    df_YoY=df[['location_id','zip_cd','week_end_dt','DMA','Q1 Campaign Type','sales_weekly_2017','sales_weekly_2018']].drop_duplicates()
    result=pd.merge(df_YoY,df_weighted_rank,on=["location_id","week_end_dt"],how="left")
    result=pd.merge(result,df_weighted_severity,on=["location_id","week_end_dt"],how="left")
    result['YoY_by_week']=(result['sales_weekly_2018']-result['sales_weekly_2017'])/result['sales_weekly_2017']
    result['location_id']=result['location_id'].astype(int)
    result=result.sort_values(['location_id','week_end_dt'])
    return result

In [52]:
agg_week_rain_desc=agg_to_weekly(df_rain_desc_long_dedup)
agg_week_snow_desc=agg_to_weekly(df_snow_desc_long_dedup)
agg_week_weather_group=agg_to_weekly(df_weather_group_long_dedup)

In [53]:
def agg_to_Q1(df):
    # all 13 weeks
    df_weighted_rank=df.groupby(["location_id"])['weighted_2_rank'].sum().to_frame().reset_index()
    df_weighted_severity=df.groupby(["location_id"])['weighted_2_severity'].sum().to_frame().reset_index()
    df_YoY=df.groupby(['location_id','zip_cd','DMA','Q1 Campaign Type'])['sales_weekly_2017','sales_weekly_2018'].sum().reset_index()
    result=pd.merge(df_YoY,df_weighted_rank,on=["location_id"],how="left")
    result=pd.merge(result,df_weighted_severity,on=["location_id"],how="left")
    result['YoY_by_week']=(result['sales_weekly_2018']-result['sales_weekly_2017'])/result['sales_weekly_2017']
    result['location_id']=result['location_id'].astype(int)
    result=result.sort_values('location_id')
    return result

In [54]:
agg_Q1_rain_desc=agg_to_Q1(df_rain_desc_long_dedup)
agg_Q1_snow_desc=agg_to_Q1(df_snow_desc_long_dedup)
agg_Q1_weather_group=agg_to_Q1(df_weather_group_long_dedup)

In [55]:
writer_Bruce=pd.ExcelWriter(writer_folder+"BL_Q1 weather aggregation.xlsx",engine="xlsxwriter")
agg_Q1_rain_desc.to_excel(writer_Bruce,"agg_Q1_rain_desc",index=False)
agg_Q1_snow_desc.to_excel(writer_Bruce,"agg_Q1_snow_desc",index=False)
agg_Q1_weather_group.to_excel(writer_Bruce,"agg_Q1_weather_group",index=False)

agg_week_rain_desc.to_excel(writer_Bruce,"agg_week_rain_desc",index=False)
agg_week_snow_desc.to_excel(writer_Bruce,"agg_week_snow_desc",index=False)
agg_week_weather_group.to_excel(writer_Bruce,"agg_week_weather_group",index=False)
writer_Bruce.save()

In [108]:
weather_joint_score[weather_joint_score['joint_weather_score']<1].shape

(1126, 8)

In [129]:
weather_joint_score=agg_week_weather_group[['week_end_dt','location_id','weighted_2_severity']]
weather_joint_score=weather_joint_score.rename(columns={"weighted_2_severity":"joint_weather_score"})

rain_joint_score=agg_week_rain_desc[['week_end_dt','location_id','weighted_2_severity']]
rain_joint_score=rain_joint_score.rename(columns={"weighted_2_severity":"rain_score"})

snow_joint_score=agg_week_snow_desc[['week_end_dt','location_id','weighted_2_severity',
                                    'sales_weekly_2017','sales_weekly_2018']]
snow_joint_score=snow_joint_score.rename(columns={"weighted_2_severity":"snow_score"})

weather_joint_score=pd.merge(weather_joint_score,rain_joint_score,on=['week_end_dt','location_id'])
weather_joint_score=pd.merge(weather_joint_score,snow_joint_score,on=['week_end_dt','location_id'])
weather_joint_score.to_csv(writer_folder+"BL_joint weather data to create matrix_JL_"+str(datetime.datetime.now().date())+".csv",index=False)




In [130]:
weather_joint_score.head(2)

,week_end_dt,location_id,joint_weather_score,rain_score,snow_score,sales_weekly_2017,sales_weekly_2018
0,2018-02-10,1,2.801694,0.599658,0.601018,65529.31,63152.41
1,2018-02-17,1,3.072440,1.155775,0.399772,72052.29,65248.17


In [131]:
writer_excel_tabs=pd.ExcelWriter(writer_folder+"BL_joint weather bin tabs"+str(datetime.datetime.now().date())+".xlsx",engine='xlsxwriter')
# Use the 13 weeks overal jont_weather_score percentile to create 5 bins
weather_joint_score['joint_score_bin']=np.where(weather_joint_score['joint_weather_score']>=np.percentile(weather_joint_score['joint_weather_score'],80),"Severity_5",
                                               np.where(weather_joint_score['joint_weather_score']>=np.percentile(weather_joint_score['joint_weather_score'],60),"Severity_4",
                                                    np.where(weather_joint_score['joint_weather_score']>=np.percentile(weather_joint_score['joint_weather_score'],40),"Severity_3",
                                                        np.where(weather_joint_score['joint_weather_score']>=np.percentile(weather_joint_score['joint_weather_score'],20),"Severity_2",
                                                            'Severity_1')
                                                            )
                                                       )
                                               )
tab_frame=pd.DataFrame({"joint_score_bin":['Severity_1','Severity_2','Severity_3','Severity_4','Severity_5']},index=[x for x in range(5)])
k=0
for week_end_dt,group in weather_joint_score.groupby(['week_end_dt']):
    #1
    df=group[['location_id','joint_score_bin','week_end_dt']]

    df_store_count=df.groupby(['joint_score_bin'])['location_id'].count().to_frame().reset_index()
    df_store_count=df_store_count.rename(columns={"location_id":str(week_end_dt)})
    
    df_store_list=df.groupby(['joint_score_bin'])['location_id'].apply(list).to_frame().reset_index()
    df_store_list=df_store_list.rename(columns={"location_id":str(week_end_dt)})
    #2
    df_score=group[['joint_weather_score','joint_score_bin','week_end_dt']]
    
    df_score_sum=df_score.groupby(['joint_score_bin'])['joint_weather_score'].sum().to_frame().reset_index()
    df_score_sum=df_score_sum.rename(columns={"joint_weather_score":str(week_end_dt)})
    
    df_score_avg=df_score.groupby(['joint_score_bin'])['joint_weather_score'].mean().to_frame().reset_index()
    df_score_avg=df_score_avg.rename(columns={"joint_weather_score":str(week_end_dt)})
    
    df_score_list=df_score.groupby(['joint_score_bin'])['joint_weather_score'].apply(list).to_frame().reset_index()
    df_score_list=df_score_list.rename(columns={"joint_weather_score":str(week_end_dt)})
    #3
    df_sales=group[['sales_weekly_2017','sales_weekly_2018','joint_score_bin','week_end_dt']]
    
    df_sales_2017=df_sales.groupby(['joint_score_bin'])['sales_weekly_2017'].sum().to_frame().reset_index()
    df_sales_2017=df_sales_2017.rename(columns={"sales_weekly_2017":str(week_end_dt)})
    
    df_sales_2018=df_sales.groupby(['joint_score_bin'])['sales_weekly_2018'].sum().to_frame().reset_index()
    df_sales_2018=df_sales_2018.rename(columns={"sales_weekly_2018":str(week_end_dt)})
    
    
    
    if k==0:
        tab_store_count=pd.merge(tab_frame,df_store_count,on="joint_score_bin",how="left")
        tab_store_list=pd.merge(tab_frame,df_store_list,on="joint_score_bin",how="left")
        tab_score_sum=pd.merge(tab_frame,df_score_sum,on="joint_score_bin",how="left")
        tab_score_avg=pd.merge(tab_frame,df_score_avg,on="joint_score_bin",how="left")
        tab_score_list=pd.merge(tab_frame,df_score_list,on="joint_score_bin",how="left")
        tab_sales_2017=pd.merge(tab_frame,df_sales_2017,on="joint_score_bin",how="left")
        tab_sales_2018=pd.merge(tab_frame,df_sales_2018,on="joint_score_bin",how="left")
        
    else:
        tab_store_count=pd.merge(tab_store_count,df_store_count,on="joint_score_bin",how="left")
        tab_store_list=pd.merge(tab_store_list,df_store_list,on="joint_score_bin",how="left")
        tab_score_sum=pd.merge(tab_score_sum,df_score_sum,on="joint_score_bin",how="left")
        tab_score_avg=pd.merge(tab_score_avg,df_score_avg,on="joint_score_bin",how="left")
        tab_score_list=pd.merge(tab_score_list,df_score_list,on="joint_score_bin",how="left")
        tab_sales_2017=pd.merge(tab_sales_2017,df_sales_2017,on="joint_score_bin",how="left")
        tab_sales_2018=pd.merge(tab_sales_2018,df_sales_2018,on="joint_score_bin",how="left")
        
    k=k+1
tab_sales_YoY=tab_sales_2017.copy()
for i in range(len(tab_sales_2018)):
    for j in range(1,tab_sales_2018.shape[1]):
        tab_sales_YoY.iloc[i,j]=(tab_sales_2018.iloc[i,j]-tab_sales_2017.iloc[i,j])/tab_sales_2017.iloc[i,j]
        
tab_store_count.to_excel(writer_excel_tabs,"tab_store_count",index=False)
tab_store_list.to_excel(writer_excel_tabs,"tab_store_list",index=False)
tab_score_sum.to_excel(writer_excel_tabs,"joint_weather_score_sum",index=False)
tab_score_avg.to_excel(writer_excel_tabs,"joint_weather_score_avg",index=False)
tab_score_list.to_excel(writer_excel_tabs,"joint_weather_score_list",index=False)
tab_sales_2017.to_excel(writer_excel_tabs,"sales_2017_sum",index=False)
tab_sales_2018.to_excel(writer_excel_tabs,"sales_2018_sum",index=False)
tab_sales_YoY.to_excel(writer_excel_tabs,"Q1_sales_YoY",index=False)
writer_excel_tabs.save()


In [132]:
tab_sales_2017.head(2)

,joint_score_bin,2018-02-10,2018-02-17,2018-02-24,2018-03-03,2018-03-10,2018-03-17,2018-03-24,2018-03-31,2018-04-07,2018-04-14,2018-04-21,2018-04-28,2018-05-05
0,Severity_1,9467267.80,11480055.93,20958127.32,822711.03,25612909.0,25048291.02,18175136.73,25398323.37,23496063.98,34059408.63,3525406.54,21597513.18,38083220.74
1,Severity_2,7194888.56,12474278.63,19478614.96,30172171.13,23559537.9,11567865.82,17016245.91,23439362.19,16440673.39,27602285.21,26275629.35,18860569.06,21691325.81


In [ ]:
Done of here

# Append YoY

## Not weighted

In [51]:
delta_temp_YoY=df_output_temp_store[['location_id','Date','Diff_Temp']].pivot(index="location_id",columns="Date",values="Diff_Temp").reset_index()
delta_temp_Max_YoY=df_output_temp_store[['location_id','Date','Diff_Temp_Max']].pivot(index="location_id",columns="Date",values="Diff_Temp_Max").reset_index()
delta_temp_Min_YoY=df_output_temp_store[['location_id','Date','Diff_Temp_Min']].pivot(index="location_id",columns="Date",values="Diff_Temp_Min").reset_index()

delta_temp_YoY=pd.merge(delta_temp_YoY,inclusion_YoY,on="location_id",how="left")
delta_temp_Max_YoY=pd.merge(delta_temp_Max_YoY,inclusion_YoY,on="location_id",how="left")
delta_temp_Min_YoY=pd.merge(delta_temp_Min_YoY,inclusion_YoY,on="location_id",how="left")

rain_YoY=df_output_rain_store[["location_id","Date","Severity"]].pivot(index="location_id",columns="Date",values="Severity").reset_index()
snow_YoY=df_output_snow_store[["location_id","Date","Severity"]].pivot(index="location_id",columns="Date",values="Severity").reset_index()
rain_YoY=pd.merge(rain_YoY,inclusion_YoY,on="location_id",how="left")
snow_YoY=pd.merge(snow_YoY,inclusion_YoY,on="location_id",how="left")

Weather_group_YoY=df_output_group_store[["location_id","Date","Severity"]].pivot(index="location_id",columns="Date",values="Severity").reset_index()
Weather_group_YoY=pd.merge(Weather_group_YoY,inclusion_YoY,on="location_id",how="left")

writer=pd.ExcelWriter(writer_folder+"Q1_inclusion_store_all_weather_type_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

delta_temp_YoY.to_excel(writer,"delta_temp_YoY",index=False)
delta_temp_Max_YoY.to_excel(writer,"delta_temp_Max_YoY",index=False)
delta_temp_Min_YoY.to_excel(writer,"delta_temp_Min_YoY",index=False)

rain_YoY.to_excel(writer,"rain_YoY",index=False)
snow_YoY.to_excel(writer,"snow_YoY",index=False)
Weather_group_YoY.to_excel(writer,"weather_group_YoY",index=False)
writer.save()

## Weighted

In [344]:
# New method to use the weekday average
weighted_delta_temp_YoY=df_output_temp_store_weight[['location_id','Date','Weighted_Diff_Temp']].pivot(index="location_id",columns="Date",values="Weighted_Diff_Temp").reset_index()
weighted_delta_temp_Max_YoY=df_output_temp_store_weight[['location_id','Date','Weighted_Diff_Temp_Max']].pivot(index="location_id",columns="Date",values="Weighted_Diff_Temp_Max").reset_index()
weighted_delta_temp_Min_YoY=df_output_temp_store_weight[['location_id','Date','Weighted_Diff_Temp_Min']].pivot(index="location_id",columns="Date",values="Weighted_Diff_Temp_Min").reset_index()

weighted_delta_temp_YoY=pd.merge(weighted_delta_temp_YoY,inclusion_YoY,on="location_id",how="left")
weighted_delta_temp_Max_YoY=pd.merge(weighted_delta_temp_Max_YoY,inclusion_YoY,on="location_id",how="left")
weighted_delta_temp_Min_YoY=pd.merge(weighted_delta_temp_Min_YoY,inclusion_YoY,on="location_id",how="left")

In [345]:
weighted_rain_YoY=df_output_rain_store_weight[["location_id","Date","Weighted_Severity"]].pivot(index="location_id",columns="Date",values="Weighted_Severity").reset_index()
weighted_snow_YoY=df_output_snow_store_weight[["location_id","Date","Weighted_Severity"]].pivot(index="location_id",columns="Date",values="Weighted_Severity").reset_index()
weighted_rain_YoY=pd.merge(weighted_rain_YoY,inclusion_YoY,on="location_id",how="left")
weighted_snow_YoY=pd.merge(weighted_snow_YoY,inclusion_YoY,on="location_id",how="left")

In [346]:
weighted_Weather_group_YoY=df_output_group_store_weight[["location_id","Date","Weighted_Severity"]].pivot(index="location_id",columns="Date",values="Weighted_Severity").reset_index()
weighted_Weather_group_YoY=pd.merge(weighted_Weather_group_YoY,inclusion_YoY,on="location_id",how="left")


In [347]:
writer=pd.ExcelWriter(writer_folder+"Q1_weighted_inclusion_store_all_weather_type_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

weighted_delta_temp_YoY.to_excel(writer,"delta_temp_YoY",index=False)
weighted_delta_temp_Max_YoY.to_excel(writer,"delta_temp_Max_YoY",index=False)
weighted_delta_temp_Min_YoY.to_excel(writer,"delta_temp_Min_YoY",index=False)

weighted_rain_YoY.to_excel(writer,"rain_YoY",index=False)
weighted_snow_YoY.to_excel(writer,"snow_YoY",index=False)
weighted_Weather_group_YoY.to_excel(writer,"weather_group_YoY",index=False)
writer.save()

In [221]:
'''
for date,group in df_output_temp_store_weight.groupby("Date"):
    df=group[["Date","location_id","Diff_Temp","Diff_Temp_Max","Diff_Temp_Min"]]
    df_95_temp=df[df['Diff_Temp']>=np.percentile(df['Diff_Temp'],95)]
    df_75_temp=df[(df['Diff_Temp']>=np.percentile(df['Diff_Temp'],75))&(df['Diff_Temp']<np.percentile(df['Diff_Temp'],95))]
    df_50_temp=df[(df['Diff_Temp']>=np.percentile(df['Diff_Temp'],50))&(df['Diff_Temp']<np.percentile(df['Diff_Temp'],75))]
    df_25_temp=df[(df['Diff_Temp']>=np.percentile(df['Diff_Temp'],25))&(df['Diff_Temp']<np.percentile(df['Diff_Temp'],50))]
    df_5_temp=df[df['Diff_Temp']<np.percentile(df['Diff_Temp'],5)]
'''

,zip_cd,Date,Collected,Temp,Temp_Max,Temp_Min,location_id,Avg_Temp,Avg_Temp_Max,Avg_Temp_Min,Diff_Temp,Diff_Temp_Max,Diff_Temp_Min,week_end_dt,Traffic,weekly_traffic,Weight_by_Traffic,Weighted_Diff_Temp,Weighted_Diff_Temp_Max,Weighted_Diff_Temp_Min
0,43232,2018-02-22,Recorded,46.004,48.2,44.6,1,44.580071,46.592857,42.414286,1.423929,1.607143,2.185714,2018-02-24,681.0,4479.0,0.152043,0.216498,0.244355,0.332322
1,30906,2018-02-22,Recorded,79.502,80.6,77.0,3,66.513071,69.221429,63.757143,12.988929,11.378571,13.242857,2018-02-24,723.0,5346.0,0.135241,1.756640,1.538853,1.790981


In [405]:
date_week_end_df=df_output_rain_store_weight[['Date','week_end_dt']].drop_duplicates()
date_week_end_df=date_week_end_df[~pd.isnull(date_week_end_df['week_end_dt'])]

In [408]:
date_week_end_df.head(2)

,Date,week_end_dt
0,2018-02-22,2018-02-24
1382,2018-03-19,2018-03-24


In [422]:
def Percentile_Value_With_0(data_frame,weather):
    k_date=0
    result_p=pd.DataFrame()

    for date,group in data_frame.groupby("Date"):
        df=group[['zip_cd','Date','Collected','location_id','Weighted_Severity']]
        df=df[df['Collected']=="Recorded"]
        P100=np.nanpercentile(df['Weighted_Severity'],100)
        P95=np.nanpercentile(df['Weighted_Severity'],95)
        P75=np.nanpercentile(df['Weighted_Severity'],75)
        P50=np.nanpercentile(df['Weighted_Severity'],50)
        P25=np.nanpercentile(df['Weighted_Severity'],25)
        P5=np.nanpercentile(df['Weighted_Severity'],5)
        P0=np.nanpercentile(df['Weighted_Severity'],0)

        df_app=pd.DataFrame({"Date":date,weather+"_Severity_Max":P100,weather+"_Severity_P95":P95,weather+"_Severity_P75":P75,
                            weather+"_Severity_P50":P50,weather+"_Severity_P25":P25,weather+"_Severity_P5":P5,weather+"_Severity_Min":P0},
                            index=[k_date])
        
        k_date=k_date+1
        result_p=result_p.append(df_app)
    result_p=pd.merge(result_p,date_week_end_df,on="Date",how="left")
    result_p=result_p[['Date',weather+"_Severity_Max",weather+"_Severity_P95",weather+"_Severity_P75",
                          weather+"_Severity_P50",weather+"_Severity_P25",weather+"_Severity_P5",weather+"_Severity_Min","week_end_dt"]]
        
    return result_p

In [423]:
rain_chart_With_0=Percentile_Value_With_0(df_output_rain_store_weight,"Rain")
snow_chart_With_0=Percentile_Value_With_0(df_output_snow_store_weight,"Snow")
group_chart_With_0=Percentile_Value_With_0(df_output_group_store_weight,"Group")

In [424]:
snow_chart_With_0.head(2)

,Date,Snow_Severity_Max,Snow_Severity_P95,Snow_Severity_P75,Snow_Severity_P50,Snow_Severity_P25,Snow_Severity_P5,Snow_Severity_Min,week_end_dt
0,2018-02-04,0.462653,0.313744,0.18295,0.0,0.0,0.0,0.0,2018-02-10
1,2018-02-05,0.355082,0.221446,0.00000,0.0,0.0,0.0,0.0,2018-02-10


In [419]:
def Percentile_Value_No_0(data_frame,weather):
    k_date=0
    result_p=pd.DataFrame()

    for date,group in data_frame.groupby("Date"):
        df=group[['zip_cd','Date','Collected','location_id','Weighted_Severity']]
        df=df[df['Collected']=="Recorded"]
        df=df[df['Weighted_Severity']>0]
        P100=np.nanpercentile(df['Weighted_Severity'],100)
        P95=np.nanpercentile(df['Weighted_Severity'],95)
        P75=np.nanpercentile(df['Weighted_Severity'],75)
        P50=np.nanpercentile(df['Weighted_Severity'],50)
        P25=np.nanpercentile(df['Weighted_Severity'],25)
        P5=np.nanpercentile(df['Weighted_Severity'],5)
        P0=np.nanpercentile(df['Weighted_Severity'],0)

        df_app=pd.DataFrame({"Date":date,weather+"_Severity_Max":P100,weather+"_Severity_P95":P95,weather+"_Severity_P75":P75,
                            weather+"_Severity_P50":P50,weather+"_Severity_P25":P25,weather+"_Severity_P5":P5,weather+"_Severity_Min":P0},
                            index=[k_date])
        
        k_date=k_date+1
        result_p=result_p.append(df_app)
    result_p=pd.merge(result_p,date_week_end_df,on="Date",how="left")
    result_p=result_p[['Date',weather+"_Severity_Max",weather+"_Severity_P95",weather+"_Severity_P75",
                          weather+"_Severity_P50",weather+"_Severity_P25",weather+"_Severity_P5",weather+"_Severity_Min","week_end_dt"]]
    return result_p

In [420]:
rain_chart_No_0=Percentile_Value_No_0(df_output_rain_store_weight,"Rain")
snow_chart_No_0=Percentile_Value_No_0(df_output_snow_store_weight,"Snow")
group_chart_No_0=Percentile_Value_No_0(df_output_group_store_weight,"Group")

In [425]:
writer_chart_P=pd.ExcelWriter(writer_folder+"Q1_Weather_Percentile_Chart_by_day_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
rain_chart_No_0.to_excel(writer_chart_P,"Rain_No_0",index=False)
snow_chart_No_0.to_excel(writer_chart_P,"Snow_No_0",index=False)
group_chart_No_0.to_excel(writer_chart_P,"group_No_0",index=False)
rain_chart_With_0.to_excel(writer_chart_P,"Rain_With_0",index=False)
snow_chart_With_0.to_excel(writer_chart_P,"Snow_With_0",index=False)
group_chart_With_0.to_excel(writer_chart_P,"group_With_0",index=False)
writer_chart_P.save()

In [387]:
sales_long=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2018-06-02.csv",dtype=str)
sales_long=sales_long[['location_id','week_end_date','sales']].rename(columns={"week_end_date":"week_end_dt"})
sales_long['location_id']=sales_long['location_id'].astype(str)
sales_long['sales']=sales_long['sales'].astype(float)
sales_long['week_end_dt']=sales_long['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())

In [388]:
sales_long_2018Q1=sales_long[(sales_long['week_end_dt']>=datetime.datetime(2018,2,10).date()) & (sales_long['week_end_dt']<=datetime.datetime(2018,5,5).date())]
sales_long_2017Q1=sales_long[(sales_long['week_end_dt']>=datetime.datetime(2017,2,11).date()) & (sales_long['week_end_dt']<=datetime.datetime(2017,5,6).date())]

sales_long_2017Q1=sales_long_2017Q1.rename(columns={"sales":"weekly_sales_2017"})
sales_long_2018Q1=sales_long_2018Q1.rename(columns={"sales":"weekly_sales_2018"})

sales_long_2017Q1['week_end_dt']=sales_long_2017Q1['week_end_dt'].apply(lambda x: x+datetime.timedelta(days=52*7))

sales_long_weekly=pd.merge(sales_long_2018Q1,sales_long_2017Q1,on=["location_id","week_end_dt"],how="inner")

In [389]:
sales_long_weekly['YoY']=(sales_long_weekly['weekly_sales_2018']-sales_long_weekly['weekly_sales_2017'])/sales_long_weekly['weekly_sales_2017']




In [390]:
sales_long_weekly.head(2)

,location_id,week_end_dt,weekly_sales_2018,weekly_sales_2017,YoY
0,1,2018-02-10,63152.41,65529.31,-0.036272
1,1,2018-02-17,65248.17,72052.29,-0.094433


In [391]:
rolledup_for_rain_weighted_severity=df_output_rain_store_weight.groupby(['location_id','week_end_dt'])['Weighted_Severity'].mean().to_frame().reset_index()
rolledup_for_rain_weighted_severity=pd.merge(rolledup_for_rain_weighted_severity,sales_long_weekly,on=["location_id","week_end_dt"],how="left")

rolledup_for_snow_weighted_severity=df_output_snow_store_weight.groupby(['location_id','week_end_dt'])['Weighted_Severity'].mean().to_frame().reset_index()
rolledup_for_snow_weighted_severity=pd.merge(rolledup_for_snow_weighted_severity,sales_long_weekly,on=["location_id","week_end_dt"],how="left")

rolledup_for_group_weighted_severity=df_output_group_store_weight.groupby(['location_id','week_end_dt'])['Weighted_Severity'].mean().to_frame().reset_index()
rolledup_for_group_weighted_severity=pd.merge(rolledup_for_group_weighted_severity,sales_long_weekly,on=["location_id","week_end_dt"],how="left")

In [392]:
writer=pd.ExcelWriter(writer_folder+"Q1_weighted_inclusion_store_by_week_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

rolledup_for_rain_weighted_severity.to_excel(writer,"rain",index=False)
rolledup_for_snow_weighted_severity.to_excel(writer,"snow",index=False)
rolledup_for_group_weighted_severity.to_excel(writer,"group",index=False)
writer.save()

In [393]:
writer=pd.ExcelWriter(writer_folder+"Q1_rain_by_week_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
rolledup_for_rain_weighted_severity.to_excel(writer,"rain_data",index=False)
for week_end in rolledup_for_rain_weighted_severity['week_end_dt'].unique():
    df=rolledup_for_rain_weighted_severity[rolledup_for_rain_weighted_severity['week_end_dt']==week_end][['location_id','week_end_dt','Weighted_Severity','YoY',"weekly_sales_2018","weekly_sales_2017"]]
    df['location_id']=df['location_id'].astype(int)
    df=df.sort_values("location_id")
    df.to_excel(writer,str(week_end),index=False)
writer.save()   
    
    

In [394]:
writer=pd.ExcelWriter(writer_folder+"Q1_snow_by_week_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
rolledup_for_snow_weighted_severity.to_excel(writer,"snow_data",index=False)
for week_end in rolledup_for_snow_weighted_severity['week_end_dt'].unique():
    df=rolledup_for_snow_weighted_severity[rolledup_for_snow_weighted_severity['week_end_dt']==week_end][['location_id','week_end_dt','Weighted_Severity','YoY',"weekly_sales_2018","weekly_sales_2017"]]
    df['location_id']=df['location_id'].astype(int)
    df=df.sort_values("location_id")
    df.to_excel(writer,str(week_end),index=False)
writer.save()   
    
    

In [395]:
writer=pd.ExcelWriter(writer_folder+"Q1_group_by_week_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
rolledup_for_group_weighted_severity.to_excel(writer,"group_data",index=False)
for week_end in rolledup_for_group_weighted_severity['week_end_dt'].unique():
    df=rolledup_for_group_weighted_severity[rolledup_for_group_weighted_severity['week_end_dt']==week_end][['location_id','week_end_dt','Weighted_Severity','YoY',"weekly_sales_2018","weekly_sales_2017"]]
    df['location_id']=df['location_id'].astype(int)
    df=df.sort_values("location_id")
    df.to_excel(writer,str(week_end),index=False)
writer.save() 
    


In [396]:
len(rolledup_for_group_weighted_severity['week_end_dt'].unique())

13

In [397]:
df_output_rain_store_weight_weekly_sales=pd.merge(df_output_rain_store_weight,sales_long_weekly,on=["location_id","week_end_dt"],how="left")
df_output_snow_store_weight_weekly_sales=pd.merge(df_output_snow_store_weight,sales_long_weekly,on=["location_id","week_end_dt"],how="left")
df_output_group_store_weight_weekly_sales=pd.merge(df_output_group_store_weight,sales_long_weekly,on=["location_id","week_end_dt"],how="left")

In [398]:

agg_count_rain_store_weight_weekly_sales=df_output_rain_store_weight_weekly_sales[df_output_rain_store_weight_weekly_sales['Collected']=="Recorded"][['location_id','Date','Severity']]
agg_count_rain_store_weight_weekly_sales=agg_count_rain_store_weight_weekly_sales.groupby(['Date','Severity'])['location_id'].count().to_frame().reset_index()
agg_count_rain_store_weight_weekly_sales=agg_count_rain_store_weight_weekly_sales.rename(columns={"location_id":"store_count"})

agg_YoY_rain_store_weight_weekly_sales=df_output_rain_store_weight_weekly_sales[df_output_rain_store_weight_weekly_sales['Collected']=="Recorded"][['location_id','week_end_dt','weekly_sales_2018','weekly_sales_2017']].drop_duplicates()
agg_YoY_rain_store_weight_weekly_sales=agg_YoY_rain_store_weight_weekly_sales.groupby(["week_end_dt","Severity"])["weekly_sales_2018","weekly_sales_2017"].sum().to_frame().reset_index()



KeyError: 'Severity'

In [ ]:
df_output_rain_store_weight_weekly_sales.head(2)

In [ ]:
test=df_output_rain_store_weight_weekly_sales.copy()
test['rain_max']=test[ranked_rain['rain_desc'].tolist()].ma

In [ ]:
df_output_rain_store_weight_weekly_sales.columns.tolist()